# LlamaIndex Bottoms-Up Development - Evaluation Baseline

LlamaIndex provides some basic evaluation of query engines! We can setup an evaluator that will measure both hallucinations, as well as if the query was actually answered!

This is provided by two main evaluations:

- `ResponseSourceEvaluator` - uses an LLM to decide if the response is similar enough to the sources -- a good measure for hallunication detection!
- `QueryResponseEvaluator` - uses an LLM to decide if a response is similar enough to the original query -- a good measure for checking if the query was answered!

You may have noticed that we are using an LLM for this task. That means we will want to pick a powerful LLM, like GPT-4 or Claude-2.

Lastly, using these methods, we can also use the LLM to generate syntheic questions to evaluate with!

## Setup the Baseline Query Engine

### Loading our Docs

In [2]:
from dotenv import load_dotenv
import os
import openai

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

In [4]:
from llama_index.core import SimpleDirectoryReader

from utils.markdown_docs_reader import MarkdownDocsReader

def load_markdown_docs(filepath):
    """Load markdown docs from a directory, excluding all other file types."""
    loader = SimpleDirectoryReader(
        input_dir=filepath,
        required_exts=[".md"],
        file_extractor={".md": MarkdownDocsReader()},
        recursive=True
    )

    documents = loader.load_data()

    # exclude some metadata from the LLM
    for doc in documents:
        doc.excluded_llm_metadata_keys = ["File Name", "Content Type", "Header Path"]

    return documents

In [5]:
# load our documents from each folder.
# we keep them seperate for now, in order to create seperate indexes later
getting_started_docs = load_markdown_docs("../docs/getting_started")
community_docs = load_markdown_docs("../docs/community")
data_docs = load_markdown_docs("../docs/core_modules/data_modules")
agent_docs = load_markdown_docs("../docs/core_modules/agent_modules")
model_docs = load_markdown_docs("../docs/core_modules/model_modules")
query_docs = load_markdown_docs("../docs/core_modules/query_modules")
supporting_docs = load_markdown_docs("../docs/core_modules/supporting_modules")
tutorials_docs = load_markdown_docs("../docs/end_to_end_tutorials")
contributing_docs = load_markdown_docs("../docs/development")

### Create the indicies

In [6]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [7]:
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage

try:
    getting_started_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./getting_started_index"))
    community_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./community_index"))
    data_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./data_index"))
    agent_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./agent_index"))
    model_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./model_index"))
    query_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./query_index"))
    supporting_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./supporting_index"))
    tutorials_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./tutorials_index"))
    contributing_index = load_index_from_storage(StorageContext.from_defaults(persist_dir="./contributing_index"))
except Exception as e:
    print(e)
    print("Creating new indices")

    getting_started_index = VectorStoreIndex.from_documents(getting_started_docs)
    getting_started_index.storage_context.persist(persist_dir="./getting_started_index")

    community_index = VectorStoreIndex.from_documents(community_docs)
    community_index.storage_context.persist(persist_dir="./community_index")

    data_index = VectorStoreIndex.from_documents(data_docs)
    data_index.storage_context.persist(persist_dir="./data_index")

    agent_index = VectorStoreIndex.from_documents(agent_docs)
    agent_index.storage_context.persist(persist_dir="./agent_index")

    model_index = VectorStoreIndex.from_documents(model_docs)
    model_index.storage_context.persist(persist_dir="./model_index")

    query_index = VectorStoreIndex.from_documents(query_docs)
    query_index.storage_context.persist(persist_dir="./query_index")

    supporting_index = VectorStoreIndex.from_documents(supporting_docs)
    supporting_index.storage_context.persist(persist_dir="./supporting_index")

    tutorials_index = VectorStoreIndex.from_documents(tutorials_docs)
    tutorials_index.storage_context.persist(persist_dir="./tutorials_index")

    contributing_index = VectorStoreIndex.from_documents(contributing_docs)
    contributing_index.storage_context.persist(persist_dir="./contributing_index")

### Create Query Engine Tools

Since we have so many indicies, we can create a query engine tool for each and then use them in a single query engine!

In [8]:
from llama_index.core.tools import QueryEngineTool

# create a query engine tool for each folder
getting_started_tool = QueryEngineTool.from_defaults(
    query_engine=getting_started_index.as_query_engine(),
    name="Getting Started",
    description="Useful for answering questions about installing and running llama index, as well as basic explanations of how llama index works."
)

community_tool = QueryEngineTool.from_defaults(
    query_engine=community_index.as_query_engine(),
    name="Community",
    description="Useful for answering questions about integrations and other apps built by the community."
)

data_tool = QueryEngineTool.from_defaults(
    query_engine=data_index.as_query_engine(),
    name="Data Modules",
    description="Useful for answering questions about data loaders, documents, nodes, and index structures."
)

agent_tool = QueryEngineTool.from_defaults(
    query_engine=agent_index.as_query_engine(),
    name="Agent Modules",
    description="Useful for answering questions about data agents, agent configurations, and tools."
)

model_tool = QueryEngineTool.from_defaults(
    query_engine=model_index.as_query_engine(),
    name="Model Modules",
    description="Useful for answering questions about using and configuring LLMs, embedding modles, and prompts."
)

query_tool = QueryEngineTool.from_defaults(
    query_engine=query_index.as_query_engine(),
    name="Query Modules",
    description="Useful for answering questions about query engines, query configurations, and using various parts of the query engine pipeline."
)

supporting_tool = QueryEngineTool.from_defaults(
    query_engine=supporting_index.as_query_engine(),
    name="Supporting Modules",
    description="Useful for answering questions about supporting modules, such as callbacks, service context, and avaluation."
)

tutorials_tool = QueryEngineTool.from_defaults(
    query_engine=tutorials_index.as_query_engine(),
    name="Tutorials",
    description="Useful for answering questions about end-to-end tutorials and giving examples of specific use-cases."
)

contributing_tool = QueryEngineTool.from_defaults(
    query_engine=contributing_index.as_query_engine(),
    name="Contributing",
    description="Useful for answering questions about contributing to llama index, including how to contribute to the codebase and how to build documentation."
)

### Create Unified Query Engine

In [9]:
# needed for notebooks
import nest_asyncio
nest_asyncio.apply()

from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.response_synthesizers import get_response_synthesizer

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        getting_started_tool,
        community_tool,
        data_tool,
        agent_tool,
        model_tool,
        query_tool,
        supporting_tool,
        tutorials_tool,
        contributing_tool
    ],
    # enable this for streaming
    # response_synthesizer=get_response_synthesizer(streaming=True),
    verbose=False
)

### Test the Query Engine!

In [9]:
response = query_engine.query("How do I install llama index?")
print(str(response))

To install llama index, you can clone the repository using Git by running `git clone https://github.com/jerryjliu/llama_index.git`. After cloning the repository, you can proceed with an editable install by running `pip install -e .`. Additionally, to install optional dependencies and dependencies used for development, you should run `pip install -r requirements.txt`.


## Evaluate the Basline!

Now that we have our baseline query engine created, we can create a basic evaluation pipeline!

Our pipeline will:

- Generate a small dataset of questions
- Save/cache these questions (so we can properly compare performance later!)
- Evaluate both response quality and hallucination

To do this reliably, we need to use an LLM smarter than `gpt-3.5-turbo`, so we will setup `gpt-4` for the evaluation process!

### Generate the Dataset

In order to make the question generation more effecient, we can remove small documents and combine all documents into a giant single docoument.

I also modify the question generation prompt, to generate a single question for each chunk, along with extra context for what it is reading.

In [10]:
from llama_index.core import Document
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("../docs", recursive=True, required_exts=[".md"]).load_data()

all_text = ""

for doc in documents:
    all_text += doc.text

giant_document = Document(text=all_text)

In [13]:
import os
import random
random.seed(42)

from llama_index.core.prompts import Prompt
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import DatasetGenerator
from llama_index.core import Settings


question_dataset = []
if os.path.exists("question_dataset.txt"):
    with open("question_dataset.txt", "r") as f:
        for line in f:
            question_dataset.append(line.strip())
else:

    # generate questions
    with Settings.temporary_set(llm=OpenAI(model="gpt-4", temperature=0)):
        data_generator = DatasetGenerator.from_documents(
            [giant_document],
            text_question_template=Prompt(
                "A sample from the LlamaIndex documentation is below.\n"
                "---------------------\n"
                "{context_str}\n"
                "---------------------\n"
                "Using the documentation sample, carefully follow the instructions below:\n"
                "{query_str}"
            ),
            question_gen_query=(
                "You are an evaluator for a search pipeline. Your task is to write a single question "
                "using the provided documentation sample above to test the search pipeline. The question should "
                "reference specific names, functions, and terms. Restrict the question to the "
                "context information provided.\n"
                "Question: "
            ),
        )

    generated_questions = data_generator.generate_questions_from_nodes()

    # randomly pick 40 questions from each dataset
    generated_questions = random.sample(generated_questions, 40)
    question_dataset.extend(generated_questions)

    print(f"Generated {len(question_dataset)} questions.")

    # save the questions!
    with open("question_dataset.txt", "w") as f:
        for question in question_dataset:
            f.write(f"{question.strip()}\n")

In [14]:
print(random.sample(question_dataset, 5))

['What is the function used to specify the metadata visible to the embedding model and how can it be customized?', 'How can I convert tools to LangChain tools using the provided documentation sample?', 'What is the purpose of the "router query engine" in the LlamaIndex framework?', 'What are the different vector stores supported by LlamaIndex for use as the storage backend for `VectorStoreIndex`?', 'What is the default number of LLM calls required for the ListIndex?']


### Evaluate with the Dataset

Now that we have our dataset, let's measure performance!

#### Evaluating Response for Hallucination

In [59]:
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()
from llama_index.core import Response

def evaluate_query_engine(evaluator, query_engine, questions):
    async def run_query(query_engine, q):
        try:
            return await query_engine.aquery(q)
        except:
            return Response(response="Error, query failed.")

    total_correct = 0
    all_results = []
    for batch_size in range(0, len(questions), 5):
        batch_qs = questions[batch_size:batch_size+5]

        tasks = [run_query(query_engine, q) for q in batch_qs]
        responses = asyncio.run(asyncio.gather(*tasks))
        print(f"finished batch {(batch_size // 5) + 1} out of {len(questions) // 5}")

        for response in responses:
            eval_result = 1 if "YES" in evaluator.evaluate_response(response=response) else 0
            total_correct += eval_result
            all_results.append(eval_result)

        # helps avoid rate limits
        time.sleep(1)

    return total_correct, all_results

In [60]:
from llama_index.core.evaluation import FaithfulnessEvaluator

gpt4 = OpenAI(temperature=0, model="gpt-4")
evaluator = FaithfulnessEvaluator(llm=gpt4)

total_correct, all_results = evaluate_query_engine(evaluator, query_engine, question_dataset)

print(f"Hallucination? Scored {total_correct} out of {len(question_dataset)} questions correctly.")

finished batch 1 out of 8
finished batch 2 out of 8
finished batch 3 out of 8
finished batch 4 out of 8
finished batch 5 out of 8
finished batch 6 out of 8
finished batch 7 out of 8
finished batch 8 out of 8
Hallucination? Scored 0 out of 40 questions correctly.


#### Investigating Hallucinations

In [43]:
import numpy as np

hallucinated_questions = np.array(question_dataset)[np.array(all_results) == 0]
print(hallucinated_questions)

['What are the possible settings for the LLM and how can the user set the prompt for term extraction?'
 'How can I convert tools to LangChain tools using the provided documentation sample?'
 'What is the purpose of the `GuidancePydanticProgram` class in the LlamaIndex documentation?'
 'What is the purpose of the SubQuestionQueryEngine class in LlamaIndex?'
 'What is the purpose of the `query_wrapper_prompt` in the `HuggingFaceLLM` class?'
 'What embedding model does LlamaIndex use by default?'
 'What are the available options for the storage backend of the index store in LlamaIndex?'
 'What is the function used to specify the metadata visible to the embedding model and how can it be customized?'
 'What are the node postprocessors available in the LlamaIndex documentation?'
 'What is the purpose of the LoadAndSearchToolSpec in the LlamaIndex documentation?'
 'What is the purpose of the DEFAULT_REFINE_PROMPT_SEL_LC in the LlamaIndex documentation?'
 "What is the purpose of the `Collectio

In [44]:
response = query_engine.query('What is the purpose of the `GuidancePydanticProgram` class in the LlamaIndex documentation?')
print(str(response))
print("-----------------")
print(response.get_formatted_sources(length=1000))

The purpose of the `GuidancePydanticProgram` class in the LlamaIndex documentation is likely related to defining and managing guidance-related data structures or functionalities within the LlamaIndex system.
-----------------
> Source (Doc id: 9c4ee42c-6781-4d9d-9617-dc8b293910f4): Sub question: What is the purpose of the `GuidancePydanticProgram` class in the LlamaIndex documentation?
Response: The `GuidancePydanticProgram` class in the LlamaIndex documentation is likely used for defining and handling guidance-related data structures or functionalities within the LlamaIndex system.

> Source (Doc id: 39fc435f-2ac7-4fe2-8396-6323230e74f6): query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

import torch
from llama_index.llms import HuggingFaceLLM
llm = HuggingFaceLLM(
    context_window=4096, 
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
   

In [45]:
response = query_engine.query('What is the purpose of the `RouterQueryEngine` in LlamaIndex and how can it be used in the search pipeline?')
print(str(response))
print("-----------------")
print(response.get_formatted_sources(length=1000))

The purpose of the `RouterQueryEngine` in LlamaIndex is to handle query transformations over index structures. It can be used in the search pipeline by directing natural language queries to different query engines based on predefined rules or conditions, allowing for more flexibility and customization in handling various types of queries within the search pipeline.
-----------------
> Source (Doc id: d7982f00-177a-4a8f-b8c4-51dd6e675ae4): Sub question: What is the purpose of the `RouterQueryEngine` in LlamaIndex?
Response: The purpose of the `RouterQueryEngine` in LlamaIndex is to handle query transformations over index structures.

> Source (Doc id: f48b7a23-e17c-4bdc-bb5d-2163bfb03cf7): Sub question: How can the `RouterQueryEngine` be used in the search pipeline?
Response: The `RouterQueryEngine` can be used in the search pipeline by taking in a natural language query and directing it to different query engines based on predefined rules or conditions. This allows for more flexibility

#### Evaluating Response for Answer Quality

In [47]:
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()
from llama_index.core import Response

def evaluate_query_engine(evaluator, query_engine, questions):
    async def run_query(query_engine, q):
        try:
            return await query_engine.aquery(q)
        except:
            return Response(response="Error, query failed.")

    total_correct = 0
    all_results = []
    for batch_size in range(0, len(questions), 5):
        batch_qs = questions[batch_size:batch_size+5]

        tasks = [run_query(query_engine, q) for q in batch_qs]
        responses = asyncio.run(asyncio.gather(*tasks))
        print(f"finished batch {(batch_size // 5) + 1} out of {len(questions) // 5}")

        for question, response in zip(batch_qs, responses):
            eval_result = 1 if "YES" in evaluator.evaluate(question, response) else 0
            total_correct += eval_result
            all_results.append(eval_result)

        # helps avoid rate limits
        time.sleep(1)

    return total_correct, all_results

In [55]:
from llama_index.core.evaluation import QueryResponseEvaluator

gpt4_llm = OpenAI(model="gpt-4", temperature=0)
evaluator = QueryResponseEvaluator(llm=gpt4_llm)

total_correct, all_results = evaluate_query_engine(evaluator, query_engine, question_dataset)

print(f"Response satisfies the query? Scored {total_correct} out of {len(question_dataset)} questions correctly.")

finished batch 1 out of 8
finished batch 2 out of 8
finished batch 3 out of 8
finished batch 4 out of 8
finished batch 5 out of 8
finished batch 6 out of 8
finished batch 7 out of 8
finished batch 8 out of 8
Response satisfies the query? Scored 0 out of 40 questions correctly.


#### Investigating Incorrect Answers

In [56]:
import numpy as np

unanswered_queries = np.array(question_dataset)[np.array(all_results) == 0]
print(unanswered_queries)

['What are the possible settings for the LLM and how can the user set the prompt for term extraction?'
 'How can I convert tools to LangChain tools using the provided documentation sample?'
 'What is the purpose of the `GuidancePydanticProgram` class in the LlamaIndex documentation?'
 'What is the purpose of the SubQuestionQueryEngine class in LlamaIndex?'
 'What is the purpose of the `query_wrapper_prompt` in the `HuggingFaceLLM` class?'
 'What embedding model does LlamaIndex use by default?'
 'What are the available options for the storage backend of the index store in LlamaIndex?'
 'What is the function used to specify the metadata visible to the embedding model and how can it be customized?'
 'What are the node postprocessors available in the LlamaIndex documentation?'
 'What is the purpose of the LoadAndSearchToolSpec in the LlamaIndex documentation?'
 'What is the purpose of the DEFAULT_REFINE_PROMPT_SEL_LC in the LlamaIndex documentation?'
 "What is the purpose of the `Collectio

In [57]:
response = query_engine.query('What is the purpose of the `ReActAgent` and how can it be initialized with other agents as tools?')
print(str(response))
print("-----------------")
print(response.get_formatted_sources(length=256))

The purpose of the ReActAgent is to be initialized from a set of Tools. It can be initialized with other agents as tools by instantiating QueryEngineTool objects with the desired agents (e.g., sql_agent, gmail_agent) and their corresponding metadata, and then passing these QueryEngineTool objects as a list to the ReActAgent's from_tools method along with any other required parameters.
-----------------
> Source (Doc id: 614e60de-cbd6-4d30-bbb6-4a78c5be0788): Sub question: What is the purpose of the ReActAgent?
Response: The purpose of the ReActAgent is to be initialized from a set of Tools.

> Source (Doc id: a22f8b29-726c-4cb1-87d7-63c6136276f0): Sub question: How can the ReActAgent be initialized with other agents as tools?
Response: The ReActAgent can be initialized with other agents as tools by instantiating QueryEngineTool objects with the desired agents (e.g., sql_agent, gmail_agent) and th...

> Source (Doc id: d6f43b48-38b2-4150-9fa3-4568b0c1a031): An agent is initialized from 

In [58]:
response = query_engine.query('What is the purpose of the LoadAndSearchToolSpec in the LlamaIndex documentation?')
print(str(response))
print("-----------------")
print(response.get_formatted_sources(length=256))

The purpose of the LoadAndSearchToolSpec in the LlamaIndex documentation is to provide a high-level interface for ingesting, indexing, and querying external data. It supports various ways of querying a list index, such as embedding-based queries to fetch the top-k neighbors and keyword filtering. Additionally, LlamaIndex allows customization through swappable storage components for document stores, index stores, and vector stores, which rely on a common Key-Value store abstraction.
-----------------
> Source (Doc id: 9afb81e7-96e0-40d8-9217-6858ceb2df88): Sub question: What is the purpose of the LoadAndSearchToolSpec in the LlamaIndex documentation?
Response: The purpose of the LoadAndSearchToolSpec in the LlamaIndex documentation is to provide a high-level interface for ingesting, indexing, and querying...

> Source (Doc id: 0a8c8d8b-868c-418b-9e86-4dacf84d80d6): Sub question: How does LoadAndSearchToolSpec interact with data loaders and index structures?
Response: LoadAndSearchToolSp

# Conclusion

In this notebook, we covered several key topics!

- setting up a sub-question query engine
- generating a dataset of evaluation questions
- evaluating responses for hallucination
- evaluating responses for answer quality